<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [2]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [3]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [4]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [5]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [6]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [7]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [8]:
response = requests.get(spacex_url)

Check the content of the response


In [9]:
print(response)

<Response [200]>


You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [10]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

We should see that the request was successfull with the 200 status response code


In [11]:
response.status_code

200

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [26]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import datetime

# SpaceX APIへのリクエストを作成
spacex_url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(spacex_url)

# リクエストが成功したかどうかをチェック
if response.status_code == 200:
    # レスポンスコンテンツをJSONとしてデコードする
    data = response.json()
    # JSONデータをデータフレームに変換する
    df = pd.json_normalize(data)
    
    # データフレームの列を絞り込む
    df = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]
    
    # rowsのフィルタリング
    df = df[df['cores'].map(len) == 1]
    df = df[df['payloads'].map(len) == 1]
    
    # coresとpayloadsの要素を取り出して置き換える
    df['cores'] = df['cores'].map(lambda x: x[0])
    df['payloads'] = df['payloads'].map(lambda x: x[0])
    
    # date_utcをdatetimeデータ型に変換し、日付のみに抽出する
    df['date'] = pd.to_datetime(df['date_utc']).dt.date
    
    # 日付でフィルタリング
    df = df[df['date'] <= datetime.date(2020, 11, 13)]
    
    # データフレームの最初の5行を表示する
    print(df.head(5))
    
else:
    print("SpaceX APIからデータの取得に失敗しました。")


                     rocket                  payloads  \
0  5e9d0d95eda69955f709d1eb  5eb0e4b5b6c3bb0006eeb1e1   
1  5e9d0d95eda69955f709d1eb  5eb0e4b6b6c3bb0006eeb1e2   
3  5e9d0d95eda69955f709d1eb  5eb0e4b7b6c3bb0006eeb1e5   
4  5e9d0d95eda69955f709d1eb  5eb0e4b7b6c3bb0006eeb1e6   
5  5e9d0d95eda69973a809d1ec  5eb0e4b7b6c3bb0006eeb1e7   

                  launchpad  \
0  5e9e4502f5090995de566f86   
1  5e9e4502f5090995de566f86   
3  5e9e4502f5090995de566f86   
4  5e9e4502f5090995de566f86   
5  5e9e4501f509094ba4566f84   

                                                                                                                                                                                            cores  \
0  {'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
1  {'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'leg

Using the dataframe <code>data</code> print the first 5 rows


In [27]:
# Get the head of the dataframe
print(df.head(5))

                     rocket                  payloads  \
0  5e9d0d95eda69955f709d1eb  5eb0e4b5b6c3bb0006eeb1e1   
1  5e9d0d95eda69955f709d1eb  5eb0e4b6b6c3bb0006eeb1e2   
3  5e9d0d95eda69955f709d1eb  5eb0e4b7b6c3bb0006eeb1e5   
4  5e9d0d95eda69955f709d1eb  5eb0e4b7b6c3bb0006eeb1e6   
5  5e9d0d95eda69973a809d1ec  5eb0e4b7b6c3bb0006eeb1e7   

                  launchpad  \
0  5e9e4502f5090995de566f86   
1  5e9e4502f5090995de566f86   
3  5e9e4502f5090995de566f86   
4  5e9e4502f5090995de566f86   
5  5e9e4501f509094ba4566f84   

                                                                                                                                                                                            cores  \
0  {'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}   
1  {'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'leg

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [29]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
df_subset = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
df_subset = df_subset[df_subset['cores'].map(len) == 1]
df_subset = df_subset[df_subset['payloads'].map(len) == 1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
df_subset['cores'] = df_subset['cores'].map(lambda x: x[0])
df_subset['payloads'] = df_subset['payloads'].map(lambda x: x[0])

# We also want to convert the date_utc to a datetime datatype and then extract the date leaving the time.
df_subset['date'] = pd.to_datetime(df_subset['date_utc']).dt.date

# Using the date we will restrict the dates of the launches.
df_subset = df_subset[df_subset['date'] <= datetime.date(2020, 11, 13)]

# Print the first 5 rows of the subset.
print(df_subset.head(5))


Empty DataFrame
Columns: [rocket, payloads, launchpad, cores, flight_number, date_utc, date]
Index: []


* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [30]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [31]:
BoosterVersion

[]

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [33]:
# Convert data to a DataFrame
df = pd.DataFrame(data)

# Call getBoosterVersion
getBoosterVersion(df)

the list has now been update 


In [34]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1']

we can apply the rest of the  functions here:


In [36]:
# Convert data to a DataFrame
df = pd.DataFrame(data)

# Call getLaunchSite
getLaunchSite(df)

In [41]:
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        load = load[0]  # リストの最初の要素を取得
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

# Call getPayloadData
getPayloadData(df)

In [61]:
def getCoreData(data):
    for core in data['cores']:
        core = core[0]  # リストの最初の要素を取得
        if core['core'] != None:
            response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
            Block.append(response['block'])
            ReusedCount.append(response['reuse_count'])
            Serial.append(response['serial'])
        else:
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
        Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
        Flights.append(core['flight'])
        GridFins.append(core['gridfins'])
        Reused.append(core['reused'])
        Legs.append(core['legs'])
        LandingPad.append(core['landpad'])

# Call getCoreData
getCoreData(df)

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [47]:
# Convert data list to DataFrame
data_df = pd.DataFrame(data)

# Create launch_dict using DataFrame columns
launch_dict = {
    'FlightNumber': list(data_df['flight_number']),
    'Date': list(data_df['date_utc']),  # Use the correct column name for date
    'BoosterVersion': BoosterVersion,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'LaunchSite': LaunchSite,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    'Reused': Reused,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial,
    'Longitude': Longitude,
    'Latitude': Latitude
}

# Create a DataFrame from launch_dict
launch_df = pd.DataFrame.from_dict(launch_dict)

# Print the DataFrame
print(launch_df)


     FlightNumber                      Date BoosterVersion  PayloadMass Orbit  \
0               1  2006-03-24T22:30:00.000Z       Falcon 1         20.0   LEO   
1               2  2007-03-21T01:10:00.000Z       Falcon 1          NaN   LEO   
2               3  2008-08-03T03:34:00.000Z       Falcon 1          NaN   LEO   
3               4  2008-09-28T23:15:00.000Z       Falcon 1        165.0   LEO   
4               5  2009-07-13T03:35:00.000Z       Falcon 1        200.0   LEO   
..            ...                       ...            ...          ...   ...   
182           183  2022-09-05T02:09:00.000Z       Falcon 9      13260.0  VLEO   
183           184  2022-09-11T01:10:00.000Z       Falcon 9      13260.0  VLEO   
184           185  2022-09-17T01:05:00.000Z       Falcon 9      13260.0  VLEO   
185           186  2022-09-24T23:30:00.000Z       Falcon 9      13260.0  VLEO   
186           187  2022-10-05T16:00:00.000Z       Falcon 9          NaN   ISS   

          LaunchSite    Out

Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [49]:
# Create a DataFrame from launch_dict
launch_df = pd.DataFrame(launch_dict)

# Print the DataFrame
print(launch_df)


     FlightNumber                      Date BoosterVersion  PayloadMass Orbit  \
0               1  2006-03-24T22:30:00.000Z       Falcon 1         20.0   LEO   
1               2  2007-03-21T01:10:00.000Z       Falcon 1          NaN   LEO   
2               3  2008-08-03T03:34:00.000Z       Falcon 1          NaN   LEO   
3               4  2008-09-28T23:15:00.000Z       Falcon 1        165.0   LEO   
4               5  2009-07-13T03:35:00.000Z       Falcon 1        200.0   LEO   
..            ...                       ...            ...          ...   ...   
182           183  2022-09-05T02:09:00.000Z       Falcon 9      13260.0  VLEO   
183           184  2022-09-11T01:10:00.000Z       Falcon 9      13260.0  VLEO   
184           185  2022-09-17T01:05:00.000Z       Falcon 9      13260.0  VLEO   
185           186  2022-09-24T23:30:00.000Z       Falcon 9      13260.0  VLEO   
186           187  2022-10-05T16:00:00.000Z       Falcon 9          NaN   ISS   

          LaunchSite    Out

Show the summary of the dataframe


In [50]:
# Show the head of the DataFrame
print(launch_df.head())


   FlightNumber                      Date BoosterVersion  PayloadMass Orbit  \
0             1  2006-03-24T22:30:00.000Z       Falcon 1         20.0   LEO   
1             2  2007-03-21T01:10:00.000Z       Falcon 1          NaN   LEO   
2             3  2008-08-03T03:34:00.000Z       Falcon 1          NaN   LEO   
3             4  2008-09-28T23:15:00.000Z       Falcon 1        165.0   LEO   
4             5  2009-07-13T03:35:00.000Z       Falcon 1        200.0   LEO   

        LaunchSite    Outcome  Flights  GridFins  Reused   Legs LandingPad  \
0  Kwajalein Atoll  None None        1     False   False  False       None   
1  Kwajalein Atoll  None None        1     False   False  False       None   
2  Kwajalein Atoll  None None        1     False   False  False       None   
3  Kwajalein Atoll  None None        1     False   False  False       None   
4  Kwajalein Atoll  None None        1     False   False  False       None   

   Block  ReusedCount    Serial   Longitude  Latitude  


### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [51]:
# Filter the DataFrame to keep only Falcon 9 launches
data_falcon9 = launch_df[launch_df['BoosterVersion'] != 'Falcon 1']


Now that we have removed some values we should reset the FlgihtNumber column


In [52]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

/var/folders/ts/2rq6mksj55z0p2hvcythxhn80000gn/T/ipykernel_75799/4023201451.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
5,1,2010-06-04T18:45:00.000Z,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
6,2,2010-12-08T15:43:00.000Z,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0004,-80.577366,28.561857
7,3,2012-05-22T07:44:00.000Z,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
8,4,2012-10-08T00:35:00.000Z,Falcon 9,400.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0006,-80.577366,28.561857
9,5,2013-03-01T19:10:00.000Z,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,178,2022-09-05T02:09:00.000Z,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True ASDS,7,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,6,B1052,-80.577366,28.561857
183,179,2022-09-11T01:10:00.000Z,Falcon 9,13260.0,VLEO,KSC LC 39A,True ASDS,14,True,True,True,5e9e3033383ecb075134e7cd,5.0,13,B1058,-80.603956,28.608058
184,180,2022-09-17T01:05:00.000Z,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True ASDS,6,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,5,B1067,-80.577366,28.561857
185,181,2022-09-24T23:30:00.000Z,Falcon 9,13260.0,VLEO,CCSFS SLC 40,True ASDS,4,True,True,True,5e9e3033383ecbb9e534e7cc,5.0,0,B1072,-80.577366,28.561857


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [53]:
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass       23
Orbit              1
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        31
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [62]:
# Calculate the mean value of PayloadMass column
payload_mass_mean = data_falcon9['PayloadMass'].mean()


# Replace the np.nan values with its mean value
data_falcon9.loc[:, 'PayloadMass'] = data_falcon9['PayloadMass'].fillna(payload_mass_mean)

/var/folders/ts/2rq6mksj55z0p2hvcythxhn80000gn/T/ipykernel_75799/2322323421.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9.loc[:, 'PayloadMass'] = data_falcon9['PayloadMass'].fillna(payload_mass_mean)


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|


Copyright © 2021 IBM Corporation. All rights reserved.
